# LLM Classifiers

**Goal:** Given a sentence as input, classify it as either a prediction or non-prediction.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from classification_models import EvaluationMetric
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
print("======= LOAD DATA =======")

======= LOAD DATA =======


In [4]:
base_data_path = os.path.join(notebook_dir, '../data/')
combine_data_path = os.path.join(base_data_path, 'combined_generated_fin_phrase_bank')
X_test_set_path = os.path.join(combine_data_path, 'x_test_set-v1.csv')
y_test_set_path = os.path.join(combine_data_path, 'y_test_set-v1.csv')

In [5]:
X_test_df = DataProcessing.load_from_file(X_test_set_path, 'csv')
X_test_df.drop(columns=['Unnamed: 0'], inplace=True)
X_test_df.head(7)

,Base Sentence,Sentence Label,Embedding,Normalized Embeddings
0,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,1,[-1.22568220e-01 2.48624131e-01 -4.82655168e-02 -1.09789923e-01\n -8.58348906e-02 3.63291465e-02 -1.27964811e-02 7.17335939e-02\n -1.18364163e-01 2.27930522e+00 -3.32476139e-01 -1.46164745e-02\n 1.25499114e-01 7.25146085e-02 6.57393187e-02 -5.76901138e-02\n -2.05907249e-03 1.51917112e+00 -3.29480022e-01 -4.57408801e-02\n 2.22120117e-02 1.05339624e-01 -4.22692373e-02 -1.23566784e-01\n -5.79119613e-03 6.18153140e-02 -5.48806489e-02 2.96797305e-02\n -1.07008472e-01 -9.98812243e-02 -9.39770564e-02 3.87728140e-02\n -5.82127571e-02 7.14930072e-02 9.65250358e-02 -3.75642404e-02\n 7.18023349e-03 6.21852428e-02 7.68979266e-02 -1.69069305e-01\n 1.44475931e-03 7.71993026e-02 3.41709815e-02 -1.10992551e-01\n -1.81387588e-02 -8.61631632e-02 -2.34903637e-02 6.00658916e-02\n -3....,[-0.14791287 -0.12622888 -0.9597454 -1.5068511 -1.6452323 1.2114074\n -0.1530719 0.91836745 -2.1379716 1.7602677 -0.42004758 -0.5548834\n 1.1031741 1.2786088 1.1086601 -0.413204 0.30347356 2.9280567\n -3.59768 -0.35298532 -0.12096298 0.67678666 -0.6748305 -0.4839827\n -0.71752447 -0.10067806 1.1003746 0.07561237 -2.1959317 -2.6175833\n -1.565941 0.42524984 -0.871213 -0.02143001 0.83343935 -0.2905648\n 0.4480813 0.1682285 1.4640632 -2.2469096 -0.02232814 0.45951518\n -0.6981451 -1.4169854 -0.70628285 -1.45749 0.5126216 1.2621828\n -1.1238234 2.1485727 2.5964377 -1.829683 1.5837685 -0.04823298\n 1.5030391 0.06060527 -0.7219292 -2.4253752 -1.7692105 -0.9184389\n -0.16441473 -0.5413762 -0.07175377 1.7040485 1.2071314 -0....
1,"The American Heart Association predicts on November 1, 2029, the obesity rates at the national level may rise.",1,[-1.44551620e-01 3.84713501e-01 4.10299003e-02 -1.51987616e-02\n -1.00560952e-02 -6.73255771e-02 1.77811161e-02 4.57233824e-02\n 8.39890912e-02 2.01573181e+00 -3.67580205e-01 -5.70408031e-02\n 1.22891583e-01 6.99080601e-02 1.97983291e-02 -5.66798002e-02\n 4.76624072e-03 1.21926570e+00 -1.45055830e-01 -3.63234729e-02\n 1.35041941e-02 -3.28723826e-02 -2.80004255e-02 -1.84945613e-01\n 3.26813050e-02 8.54878053e-02 -1.49334148e-01 4.50391369e-03\n 3.68039422e-02 1.91185996e-01 2.18118466e-02 1.32588789e-01\n 8.98898989e-02 1.37999743e-01 1.21656165e-01 6.33542910e-02\n -1.06478065e-01 8.26832876e-02 -4.30808812e-02 1.46262258e-01\n -2.19029468e-02 4.71463166e-02 -2.72156689e-02 -2.27241516e-02\n 8.14019889e-02 -8.10244754e-02 -7.50189573e-02 -1.04098104e-01\n 6....,[-0.41808617 1.70707 0.48293787 0.04548473 -0.63167155 -0.43579143\n 0.42126647 0.5806888 0.5016804 0.8586611 -0.9072807 -1.092698\n 1.0650555 1.2340899 0.6353464 -0.39396188 0.42907965 0.8229404\n -0.52147204 -0.19341081 -0.3010331 -1.3742509 -0.43909413 -1.3607903\n -0.12759547 0.21505134 -0.35859266 -0.35160506 0.23767695 1.5916128\n 0.55531317 1.9194982 1.4566973 0.9181272 1.3417342 1.4131705\n -1.0620697 0.4907245 -0.60323673 2.487476 -0.44338858 -0.01337191\n -1.5358676 0.08269311 1.17976 -1.3706315 -0.20932262 -0.7469172\n 0.5761304 -0.17463458 -1.0923884 0.02983202 0.9992851 -0.7784116\n 0.1491646 0.0532804 1.4128524 1.1143323 -0.23840122 -0.9047452\n -0.31026492 1.8315382 1.4846493 0.20744875 -1.2032983 -0.0...
2,"On 2025-06-01, Meteorologist Emily Chen speculates that the temperature at Dallas will likely increase.",1,[-1.68004587e-01 3.62666279e-01 -5.09679969e-03 -1.52042462e-02\n -3.66210565e-02 -1.76503416e-02 2.71140393e-02 1.10233976e-02\n -1.17255524e-01 1.37631774e+00 -2.74166286e-01 -1.44476595e-03\n -4.53882962e-02 -9.81117859e-02 -1.77818686e-01 2.31416486e-02\n -6.66156188e-02 1.01895070e+00 -1.28533006e-01 -3.99891064e-02\n 7.87591264e-02 1.45874396e-01 1.31233614e-02 -1.26720756e-01\n 1.07190073e-01 1.09603144e-01 -1.19070552e-01 -7.85431415e-02\n 1.93831511e-02 -6.62952056e-03 -1.7837

In [6]:
y_test_df = DataProcessing.load_from_file(y_test_set_path, 'csv')
y_test_df.drop(columns=['Unnamed: 0'], inplace=True)
print(f"\t{y_test_df.head(7)}")

	   Sentence Label
0               1
1               1
2               1
3               1
4               1
5               1
6               1


## Load Prompt

In [7]:
# prediction_properties = PredictionProperties.get_prediction_properties()
# prediction_requirements = PredictionProperties.get_requirements()
# system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
# prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
#     1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
#     2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
#     3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
# """
# non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
#     1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
#     2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
#     3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
# """
# # goal_prompt = "Given the above, identify the prediction."

# base_prompt = f"""{system_identity_prompt} The sentence format is based on: 
    
#     {prediction_properties}
#     Enforce: {prediction_requirements}
#     Know: {prediction_examples_prompt}
#     Know: {non_prediction_examples_prompt}

# """
# base_prompt

In [8]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow."
- "The stock market is expected to rise next quarter."
- "I am going to the store."
- “Lakers will win the championship.”

A prediction may contain: source, target, date, outcome.
"""

## Models

In [9]:
tgmf = TextGenerationModelFactory()


# Groq Cloud (https://console.groq.com/docs/overview)
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  

# models  = [llama_318b_instant_generation_model]
models  = [llama_318b_instant_generation_model, llama_3370b_versatile_generation_model]


# NaviGator
# llama_31_70b_instruct = tgmf.create_instance('llama-3.1-70b-instruct') 
# mistral_small_31 = tgmf.create_instance('mistral-small-3.1') 
# models = [llama_31_70b_instruct, mistral_small_31]

In [10]:
import json
import re

def parse_json_response(response):
    """Parse JSON response from LLM to extract label and reasoning"""
    try:
        # Extract JSON if there's extra text
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            return data.get('label'), data.get('reasoning')
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None, None

In [11]:
def llm_certifier(data: str, base_prompt: str, model):
    
        prompt = f""" Given this: {base_prompt}. Also given the sentence '{data}', your task is to analyze the sentence and determine if it is a prediction. If prediction, generate label as 1 and if non-prediction generate label as 0.
        Respond ONLY with valid JSON in this exact format:
        {{"label": 0, "reasoning": "your explanation here"}}
        Examples:
        - "It will rain tomorrow." → {{"label": 1, "reasoning": "Contains the future tense words 'will' and 'tomorrow'"}}
        - "The stock market is expected to rise next quarter." → {{"label": 1, "reasoning": "Contains future tense words 'is expected'"}}
        - "I am going to the store." → {{"label": 0, "reasoning": "Does not contain a future tense word"}}
        - "Lakers will win the championship." → {{"label": 1, "reasoning": "Contains the future tense word 'will'"}}
        """
        idx = 1
        if idx == 1:
            #   print(f"\tPrompt: {prompt}")
              idx = idx + 1
        input_prompt = model.user(prompt)
        raw_text_llm_generation = model.chat_completion([input_prompt])
        
        # Parse the JSON response
        label, reasoning = parse_json_response(raw_text_llm_generation)
        
        return raw_text_llm_generation, label, reasoning

In [12]:
print("======= PROMPT + MODEL -> LABEL and REASONING =======")

======= PROMPT + MODEL -> LABEL and REASONING =======


In [13]:
# content : meta :: text : meta_data
results = []
for idx, row in X_test_df.iterrows():
    text = row['Base Sentence']
    print('\t', idx, text)
    for model in models:
        raw_response, llm_label, llm_reasoning = llm_certifier(text, prompt_1, model)
        print('\t\t', model.__name__(), '--- Label:', llm_label, '--- Reasoning:', llm_reasoning)
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


	 0 With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
		 llama-3.1-8b-instant --- Label: 1 --- Reasoning: Contains future tense words 'would increase', 'would improve', and 'therefore increase' which indicate a prediction about future events
		 llama-3.3-70b-versatile --- Label: 1 --- Reasoning: Contains the future tense words 'would' and implies a future outcome of increased capacity, improved use of raw materials, and increased production profitability
	 1 The American Heart Association predicts on November 1, 2029, the obesity rates at the national level may rise.
		 llama-3.1-8b-instant --- Label: 1 --- Reasoning: Contains the future tense word 'will' and a specific date 'November 1, 2029' indicating a prediction about the future
		 llama-3.3-70b-versatile --- Label: 1 --- Reasoning: Contains the future tense words 'predicts' a

In [14]:
results

[('With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .',
  '{"label": 1, "reasoning": "Contains future tense words \'would increase\', \'would improve\', and \'therefore increase\' which indicate a prediction about future events"}',
  1,
  "Contains future tense words 'would increase', 'would improve', and 'therefore increase' which indicate a prediction about future events",
  'llama-3.1-8b-instant'),
 ('With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .',
  '{"label": 1, "reasoning": "Contains the future tense words \'would\' and implies a future outcome of increased capacity, improved use of raw materials, and increased production profitability"}',
  1,
  "Contains the future t

In [15]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df

,text,raw_response,llm_label,llm_reasoning,llm_name
0,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,"{""label"": 1, ""reasoning"": ""Contains future tense words 'would increase', 'would improve', and 'therefore increase' which indicate a prediction about future events""}",1,"Contains future tense words 'would increase', 'would improve', and 'therefore increase' which indicate a prediction about future events",llama-3.1-8b-instant
1,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,"{""label"": 1, ""reasoning"": ""Contains the future tense words 'would' and implies a future outcome of increased capacity, improved use of raw materials, and increased production profitability""}",1,"Contains the future tense words 'would' and implies a future outcome of increased capacity, improved use of raw materials, and increased production profitability",llama-3.3-70b-versatile
2,"The American Heart Association predicts on November 1, 2029, the obesity rates at the national level may rise.","{""label"": 1, ""reasoning"": ""Contains the future tense word 'will' and a specific date 'November 1, 2029' indicating a prediction about the future""}",1,"Contains the future tense word 'will' and a specific date 'November 1, 2029' indicating a prediction about the future",llama-3.1-8b-instant
3,"The American Heart Association predicts on November 1, 2029, the obesity rates at the national level may rise.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'predicts' and 'may rise', and includes a specific future date 'November 1, 2029', indicating a statement about a future outcome""}",1,"Contains the future tense words 'predicts' and 'may rise', and includes a specific future date 'November 1, 2029', indicating a statement about a future outcome",llama-3.3-70b-versatile
4,"On 2025-06-01, Meteorologist Emily Chen speculates that the temperature at Dallas will likely increase.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'will likely increase' and specifies a target (temperature at Dallas) and a date (2025-06-01)""}",1,Contains the future tense words 'will likely increase' and specifies a target (temperature at Dallas) and a date (2025-06-01),llama-3.1-8b-instant
5,"On 2025-06-01, Meteorologist Emily Chen speculates that the temperature at Dallas will likely increase.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'will likely increase' and a specific date '2025-06-01', indicating a prediction about a future event""}",1,"Contains the future tense words 'will likely increase' and a specific date '2025-06-01', indicating a prediction about a future event",llama-3.3-70b-versatile
6,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.","{""label"": 1, ""reasoning"": ""Contains the future tense phrase 'envision that' and a specific date '08/2024' which indicates a prediction about the future.""}",1,Contains the future tense phrase 'envision that' and a specific date '08/2024' which indicates a prediction about the future.,llama-3.1-8b-instant
7,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.","{""label"": 1, ""reasoning"": ""Contains the future tense words 'envisions' and a specific date '08/2024', indicating a prediction about the future outcome of the touchdown rate at the New England Patriots""}",1,"Contains the future tense words 'envisions' and a specific date '08/2024', indicating a prediction about the future outcome of the touchdown rate at the New England Patriots",llama-3.3-70b-versatile
8,The World Health Organization 

In [16]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']

llama_instant_labels = get_llm_labels(results_with_llm_label_df, 'llama-3.1-8b-instant')
llama_versatile_labels = get_llm_labels(results_with_llm_label_df, 'llama-3.3-70b-versatile')
print(f"\tllama-3.1-8b-instant: {llama_instant_labels}")
print(f"\tllama-3.3-70b-versatile: {llama_versatile_labels}")

	llama-3.1-8b-instant: 0     1
2     1
4     1
6     1
8     1
10    0
12    1
14    1
16    1
18    1
20    0
22    0
24    0
26    1
28    0
30    1
32    1
34    1
36    1
38    1
40    1
Name: llm_label, dtype: int64
	llama-3.3-70b-versatile: 1     1
3     1
5     1
7     1
9     1
11    0
13    1
15    1
17    1
19    1
21    0
23    0
25    0
27    1
29    0
31    1
33    1
35    1
37    1
39    1
41    1
Name: llm_label, dtype: int64


In [17]:
model_predictions_df = pd.concat([X_test_df['Base Sentence'], y_test_df], axis=1)
model_predictions_df.columns = ['Sentence', 'Actual Label']
model_predictions_df['Instant'] = llama_instant_labels.to_numpy().ravel()
model_predictions_df['Versatile Label'] = llama_versatile_labels.to_numpy().ravel()
model_predictions_df

# print(f"{model_predictions_df}")

,Sentence,Actual Label,Instant,Versatile Label
0,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,1,1,1
1,"The American Heart Association predicts on November 1, 2029, the obesity rates at the national level may rise.",1,1,1
2,"On 2025-06-01, Meteorologist Emily Chen speculates that the temperature at Dallas will likely increase.",1,1,1
3,"In 08/2024, Coach Michael Brown envisions that the touchdown rate at the New England Patriots has some probability to remain stable.",1,1,1
4,The World Health Organization forecasts that the prevalence of chronic illnesses at urban health centers in Africa will potentially decrease in Q2 2028.,1,1,1
5,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",1,0,0
6,Business analyst John Lee forecasts that the stock prices at Tesla potentially decrease in Q3 of 2027.,1,1,1
7,"Dr. David Kim predicts on 21 August 2024, the average salary at Google may rise.",1,1,1
8,"On 08/20/2024, Analyst Kevin White speculates the win rate at the Los Angeles Rams will likely increase.",1,1,1
9,"The corporate tax rate in the healthcare sector should stay the same in 2026, according to policy analyst Emily Wong.",1,1,1


## Evaluation

In [18]:
print("======= EVALUATION/RESULTS =======")

======= EVALUATION/RESULTS =======


In [19]:
get_metrics = EvaluationMetric()
get_metrics

In [20]:
metrics = get_metrics.eval_classification_report(y_test_df, llama_instant_labels)
metrics

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.94      0.97        17

    accuracy                           0.95        21
   macro avg       0.90      0.97      0.93        21
weighted avg       0.96      0.95      0.95        21



In [21]:
metrics = get_metrics.eval_classification_report(y_test_df, llama_versatile_labels)
metrics

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.94      0.97        17

    accuracy                           0.95        21
   macro avg       0.90      0.97      0.93        21
weighted avg       0.96      0.95      0.95        21

